In [ ]:
import numpy as np


def legacy(v, lon, alpha, R, lat, elev, delta):
    def local_hour_angle(apparent_sidereal_time, observer_longitude, sun_right_ascension):
        """Measured westward from south"""
        H = apparent_sidereal_time + observer_longitude - sun_right_ascension
        return H % 360


    def equatorial_horizontal_parallax(earth_radius_vector):
        xi = 8.794 / (3600 * earth_radius_vector)
        return xi


    def uterm(observer_latitude):
        u = np.arctan(0.99664719 * np.tan(np.radians(observer_latitude)))
        return u


    def xterm(u, observer_latitude, observer_elevation):
        x = np.cos(u) + observer_elevation / 6378140 * np.cos(np.radians(observer_latitude))
        return x


    def yterm(u, observer_latitude, observer_elevation):
        y = 0.99664719 * np.sin(u) + observer_elevation / 6378140 * np.sin(np.radians(observer_latitude))
        return y

    def parallax_sun_right_ascension(
        xterm,
        equatorial_horizontal_parallax,
        local_hour_angle,
        geocentric_sun_declination,
    ):
        # ∆α = Arc tan 2 ( − x *sin ξ *sin H / cosδ − x * sin ξ * cos H)
        equatorial_horizontal_parallax_rad = np.radians(equatorial_horizontal_parallax)
        local_hour_angle_rad = np.radians(local_hour_angle)

        num = -xterm * np.sin(equatorial_horizontal_parallax_rad) * np.sin(local_hour_angle_rad)
        denom = np.cos(np.radians(geocentric_sun_declination)) - xterm * np.sin(
            equatorial_horizontal_parallax_rad
        ) * np.cos(local_hour_angle_rad)
        delta_alpha = np.degrees(np.arctan2(num, denom))
        return delta_alpha

    def topocentric_local_hour_angle(local_hour_angle, parallax_sun_right_ascension):
        H_prime = local_hour_angle - parallax_sun_right_ascension
        return H_prime
    def topocentric_sun_right_ascension(geocentric_sun_right_ascension, parallax_sun_right_ascension):
        alpha_prime = geocentric_sun_right_ascension + parallax_sun_right_ascension
        return alpha_prime


    def topocentric_sun_declination(
        geocentric_sun_declination,
        xterm,
        yterm,
        equatorial_horizontal_parallax,
        parallax_sun_right_ascension,
        local_hour_angle,
    ):
        geocentric_sun_declination_rad = np.radians(geocentric_sun_declination)
        equatorial_horizontal_parallax_rad = np.radians(equatorial_horizontal_parallax)

        num = (np.sin(geocentric_sun_declination_rad) - yterm * np.sin(equatorial_horizontal_parallax_rad)) * np.cos(
            np.radians(parallax_sun_right_ascension)
        )
        denom = np.cos(geocentric_sun_declination_rad) - xterm * np.sin(equatorial_horizontal_parallax_rad) * np.cos(
            np.radians(local_hour_angle)
        )
        delta = np.degrees(np.arctan2(num, denom))
        return delta


# def main(v, lon, alpha, R, lat, elev, delta):
    H = local_hour_angle(v, lon, alpha)
    xi = equatorial_horizontal_parallax(R)
    u = uterm(lat)
    x = xterm(u, lat, elev)
    y = yterm(u, lat, elev)
    delta_alpha = parallax_sun_right_ascension(x, xi, H, delta)
    delta_prime = topocentric_sun_declination(delta, x, y, xi, delta_alpha, H)
    H_prime = topocentric_local_hour_angle(H, delta_alpha)
    return delta_prime, H_prime



